<a href="https://colab.research.google.com/github/StefanVaylBX2023/CSE303/blob/main/New_version_for_unions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##As before:
Here nothing changes, same as before:
- parsing the input
- create the graph and traverse it
- add "free" observations

In the last step mentioned above there's a small change, becasue in the same loop i create a "metagraph" that will be exploted later when taking the unions

In [1]:
from sympy import *
import sys
import time
import itertools as it


In [2]:
def read_input(filename):
    
    with open(filename) as f:
        lines = f.readlines()
    graph = {}
    ind = {}
    count = 0
    for line in lines:
        line = line.split('=')
        line[0] = list(exp(line[0]).free_symbols)[0]
        if str(line[0]).startswith('d'):
            line[0] = str(line[0]).split('d')[1]
            line[0] = Symbol(line[0])
        graph[line[0]] = list(exp(line[1]).free_symbols)
        ind[line[0]] = count
        count += 1

    return ind, graph

def list_of_states(graph): #X = list of states, Y = list of outputs
    X = []  
    for value in graph.values():
        for i in value:
            if i not in X:
                X.append(i)

    Y = []  
    for key in graph.keys():
        if key not in Y and str(key).startswith('y'):
            Y.append(key)
    return X, Y


def dfs(graph, start, path=[]):
    path = path + [start]
    for node in graph[start]:
        if node not in path:
            path = dfs(graph, node, path)
    return path

# traverse the graph
def find_models(graph, Y): 
    models = {}
    for y in Y:
        models[y] = dfs(graph, y)
    return models


##New additions:

This function here has a little addition that is needed to build an initial version of the metagraph 

In [3]:
def find_submodels(graph, models, Y):
    metagraph = {}
    for y in Y:
        metagraph[y] = []
        for i in Y:
            if i != y:
                if set(graph[i]).issubset(set(models[y])):
                    if i not in models[y]:
                      models[y].append(i)
                      #CHANGED HERE  
                    metagraph[y].append(i)
    res = []
    for y in Y:
        res.append(models[y])
    return res, metagraph

This is the new version of the code that finds the submodels

In [18]:
def find_new_models_vers_four(models, metagraph, Y):

  #We add the model to the list of submmodels (we say that a model is
  #submodel of itself)
  for key in list(metagraph):
    metagraph[key].append(key)

  #We sort the array (even tho probably is not needed now, it was in the
  #previous version of the algorithm)
  sorted_arr = sorted(list(metagraph.values()), key=len, reverse=True)

  #Lists of lists that allows me to see if he method took all the 
  #unions that it will have to take, not returned, we can delete it.
  model_to_add_symbol = []
  #The array that will be returned and that will store the new submodels 
  model_to_add_real = []

  for element in sorted_arr:
    #we create a list of indices that we need to check (this will avoid
    #taking the union of a model with its submodels) 
    other_indices = list(set(Y)-set(element))

    #We iterate through the submodels
    for other_element in sorted_arr:
      #We check if the submodel under consideration (other_element) is not a 
      #submodel of the one we are taking the union with (element)
      if set(other_element).issubset(set(other_indices)):
        #we add the model to the list of unions we need to take
        model_to_add_symbol.append([other_element, element])
        #we take the unio of the models and we store the result
        model_to_add_real.append(list(set(models[get_key(other_element, metagraph)]).union(set(models[get_key(element, metagraph)]))))
    #We take off the element we just considered from the array on which we 
    #iterate, because any significant union with this model have been already taken
    #at this step of the algorithm 
    sorted_arr.remove(element)
  #We return the list of models that we need to add
  return model_to_add_real

#Small fucntions that given a value in a dict and its original dict
#returns the corresponding key
def get_key(val, my_dict):
    for key, value in my_dict.items():
        if val == value:
            return key
 
    return "key doesn't exist"


In [19]:
graph_4 = read_input("/content/drive/MyDrive/Bocconi/Colab Notebooks/CSE303/counterexample_1.txt")[1]
X_4, Y_4 = list_of_states(graph_4)
models_4 = find_models(graph_4, Y_4)
res_4 = find_submodels(graph_4, models_4, Y_4)[0] #Now i need the first of the two returns
metagraph_4 = find_submodels(graph_4, models_4, Y_4)[1] #We take the raw metagraph
extra_models = find_new_models_vers_four(models_4, metagraph_4, Y_4) #We find the other models
finalres = list(models_4.values()) + extra_models #We create the final list of submodels

print("The submodels are:\n")
for element in finalres:
  print(f"{element}\n")

The submodels are:

[y1, x1, x2]

[y2, x3, x2]

[y3, x4, x3, x2, y2]

[y4, x4, x3, x2, x1, y1, y2, y3]

[y5, x5]

[y1, x4, y2, x1, x5, y4, x3, y3, x2, y5]

[y1, y3, y2, x3, x4, x1, x2]

[y1, x3, y2, x1, x2]

[y1, x1, x2, x5, y5]

[y3, y2, x3, x4, x2, x5, y5]

[x3, y2, x2, x5, y5]

